# CCS potential with LAMMPS  for ZnO

In this example, we will run CCS potentials with LAMMPS for multicomponent systems. ZnO  CCS potential is used for this illustration. Note that the exponential head part in **CCS_params.json** is ignored when LAMMPS spline tables are created. The exponential part is avoided because LAMMPS performs a pre-interpolation on the tabular data, and often pre-interpolation done with exponential head gives worse results. In general, a higher resolution should be used while generating spline table for LAMMPS. By default, the gridsize used for CCS is divided by 10.

**Note:** 
* Install scikit-learn if it's not installed yet; 
https://scikit-learn.org/stable/install.html

In [1]:
# Generates CCS.table file readable by LAMMPS
from ccs_fit.scripts.jsonTotable import asecalcTotable
tags=asecalcTotable("CCS_params.json",scale=10)  # Controls the resolution of the gridsize; gridsize=dr/scale.
tags

{'O-Zn': {'Rmin': 1.2999991089415996,
  'Rcut': 5.9999991089416,
  'dr': 0.005,
  'N': 941},
 'O-O': {'Rmin': 2.0999990640749275,
  'Rcut': 5.9999990640749274,
  'dr': 0.01,
  'N': 391},
 'Zn-Zn': {'Rmin': 1.9999957089651335,
  'Rcut': 5.999995708965134,
  'dr': 0.01,
  'N': 401}}

In [2]:
!head CCS.table


 O-Zn
 N 941 R 1.2999991089415996 5.9999991089416 

 1 1.2999991089415996 18.6733869381274 391.5424306774645
 2 1.3049991089415995 16.74203855060834 380.99692432954544
 3 1.3099991089415994 14.863417694838336 370.45141797784134
 4 1.3149991089415993 13.037524370836312 359.9059116223522
 5 1.3199991089415992 11.264358578621199 349.3604052630781
 6 1.3249991089415991 9.543920318211915 338.814898900019


In [3]:
import json
import copy
import matplotlib.pyplot as plt

from ase import Atoms
from ase.calculators.lammpsrun import LAMMPS
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from ase.io import read,write
from ase.io.trajectory import Trajectory
from ase.optimize import BFGS
from sklearn.metrics import mean_squared_error
import numpy as np


with open ('CCS_params.json','r') as f:
    CCS_params=json.load(f)


# Parameters for LAMMPS 
parameters = {
              'pair_style': 'table spline 1000',   # The number of elements chosen taken for pre-interpolation
              'pair_coeff': ['1 1  CCS.table O-O 5.9999990640749274', # Specify Rcut for each pair
                             '1 2  CCS.table O-Zn 5.9999991089416',
                             '2 2  CCS.table Zn-Zn 5.999995708965134'],
#              'pair_write': ['1 2 500 r  1.2999991089415996 5.9999991089416 table.txt table']
              'command': '/usr/bin/lmp'   # Remove or change to your local lammps executable
}



# Single point evaluation

In [4]:
struct=read('TiO2.poscar')

chem_symbs = struct.get_chemical_symbols()
chem_symbs[0] = 'Zn'
chem_symbs[1] = 'Zn'

struct.set_chemical_symbols(chem_symbs)

In [5]:
from pymatgen.core import Structure

struc_pmg = Structure.from_file("TiO2.POSCAR")

print(dir(struc_pmg))

print(struc_pmg.get_space_group_info())

['DISTANCE_TOLERANCE', 'REDIRECT', '__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__modify_schema__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_charge', '_get_neighbor_list_py', '_lattice', '_sites', 'add_oxidation_state_by_element', 'add_oxidation_state_by_guess', 'add_oxidation_state_by_site', 'add_site_property', 'add_spin_by_element', 'add_spin_by_site', 'append', 'apply_operation', 'apply_strain', 'as_dataframe', 'as_dict', 'atomic_numbers', 'cart_coords', 'charge', 'clear', 'composition', 'copy', 'count'

In [6]:
# struct=read('POSCAR')
lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)
ccs= CCS(CCS_params)


struct_ase=copy.deepcopy(struct)

struct.calc = lammps
print("Energy from LAMMPS  pre optmization: ", struct.get_potential_energy())
struct_ase.calc=ccs
print("Energy from CCS calculator pre optmization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))




/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/ase/calculators/lammpsrun.py:191: UserWarning: You are using an old syntax to set 'parameters'.
Please use LAMMPSRUN.set().
  warnings.warn(self.legacy_warn_string.format("parameters"))


Energy from LAMMPS  pre optmization:  -10.40607858103671
Energy from CCS calculator pre optmization: -10.406102645641965
MSE on energy between ase calc and LAMMPS: 5.791052261298848e-10
MSE on forces between ase calc and LAMMPS: 3.610305740347139e-08


/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


# Optimization

In [7]:
traj = Trajectory('LAMMPS.traj', 'w', struct)
dyn=BFGS(struct)
dyn.attach(traj)
dyn.run(fmax=0.005)
print("Energy from LAMMPS after optimization: ", struct.get_potential_energy())


traj = Trajectory('CCS.traj', 'w', struct_ase)
dyn=BFGS(struct_ase)
dyn.attach(traj)
dyn.run(fmax=0.005)
print("Energy from CCS calculator after optimization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))



      Step     Time          Energy         fmax
BFGS:    0 13:14:39      -10.406079        1.8709
BFGS:    1 13:14:39      -10.529349        0.4050
BFGS:    2 13:14:39      -10.534953        0.0251
BFGS:    3 13:14:39      -10.534973        0.0003
Energy from LAMMPS after optimization:  -10.53497273922022
      Step     Time          Energy         fmax
BFGS:    0 13:14:39      -10.406103        1.8705
BFGS:    1 13:14:39      -10.529348        0.4055
BFGS:    2 13:14:39      -10.534951        0.0256
BFGS:    3 13:14:39      -10.534974        0.0003
Energy from CCS calculator after optimization: -10.53497420618288
MSE on energy between ase calc and LAMMPS: 2.151979445220317e-12
MSE on forces between ase calc and LAMMPS: 1.5276948768853066e-09


/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


**Note** : 
* Onebody energy contributons in CCS_params file was set to 0 for easy comparison

### Adding cell optimisation

In [8]:
from ase.constraints import StrainFilter
from ase.io import read

In [13]:
opt_struct = read('LAMMPS.traj')
opt_struct.calc = lammps

print(opt_struct.get_stress())

orig_cell = opt_struct.cell.copy()

sf = StrainFilter(opt_struct)
opt = BFGS(sf, trajectory='LAMMPS_strain.traj')
opt.run(fmax=1e-4)
alat = opt_struct.cell[0][1] - opt_struct.cell[0][0]

LAMMPS_strain_opt = read('LAMMPS_strain.traj')
LAMMPS_strain_opt_cell = LAMMPS_strain_opt.cell

old_vol = np.prod(np.diag(orig_cell))
new_vol = np.prod(np.diag(LAMMPS_strain_opt_cell))

print("Volume change of {} %, as a result of the following lattice parameter changes:".format((new_vol-old_vol)/old_vol*100))
print([np.diag(LAMMPS_strain_opt_cell-orig_cell)[i]/np.diag(orig_cell)[i]*100 for i in range(3)])

[-1.58149341e+00 -1.58149341e+00 -1.16619126e+00 -4.70736975e-16
  8.46881731e-17 -4.25081798e-16]
      Step     Time          Energy         fmax
BFGS:    0 09:38:30      -10.534973      157.9307
BFGS:    1 09:38:30      -31.201993       75.3388
BFGS:    2 09:38:30      -42.494719       78.3581
BFGS:    3 09:38:30      -52.638107       80.7954
BFGS:    4 09:38:30      -60.163204       45.1199
BFGS:    5 09:38:30      -66.646994       44.1823
BFGS:    6 09:38:30      -70.531264       34.6771
BFGS:    7 09:38:30      -71.835801       23.3584
BFGS:    8 09:38:30      -72.643322        6.0235
BFGS:    9 09:38:30      -72.722339        3.3311
BFGS:   10 09:38:30      -72.761553        0.2820
BFGS:   11 09:38:30      -72.761937        0.0224
BFGS:   12 09:38:30      -72.761939        0.0004
BFGS:   13 09:38:30      -72.761939        0.0000
Volume change of 276.0441279733187 %, as a result of the following lattice parameter changes:
[54.55774912085602, 54.55774912076761, 57.41903391072035]


In [14]:
from ase.io import write, read

opt_struct = read('CCS.traj')
opt_struct.calc = ccs

print(opt_struct.get_stress())

orig_cell = opt_struct.cell.copy()

sf = StrainFilter(opt_struct)
opt = BFGS(sf, trajectory='CCS_strain.traj')
opt.run(fmax=1e-4)
alat = opt_struct.cell[0][1] - opt_struct.cell[0][0]

CCS_strain_opt = read('CCS_strain.traj')
CCS_strain_opt_cell = CCS_strain_opt.cell

old_vol = np.prod(np.diag(orig_cell))
new_vol = np.prod(np.diag(CCS_strain_opt_cell))

write("TiO2_opt_ccs.POSCAR", CCS_strain_opt)

print("Volume change of {} %, as a result of the following lattice parameter changes:".format((new_vol-old_vol)/old_vol*100))
print([np.diag(CCS_strain_opt_cell-orig_cell)[i]/np.diag(orig_cell)[i]*100 for i in range(3)])

print(np.diag(CCS_strain_opt_cell))

[-1.58150180e+00 -1.58150180e+00 -1.16619344e+00  2.63065528e-16
  1.81991438e-16  9.50416058e-16]
      Step     Time          Energy         fmax
BFGS:    0 09:38:45      -10.534974      157.9314
BFGS:    1 09:38:45      -31.201994       75.3398
BFGS:    2 09:38:45      -42.494789       78.3584
BFGS:    3 09:38:45      -52.638099       80.7956
BFGS:    4 09:38:45      -60.163193       45.1199
BFGS:    5 09:38:45      -66.647390       44.1810
BFGS:    6 09:38:45      -70.531592       34.6746
BFGS:    7 09:38:45      -71.836017       23.3555
BFGS:    8 09:38:45      -72.643331        6.0235
BFGS:    9 09:38:45      -72.722348        3.3309
BFGS:   10 09:38:45      -72.761559        0.2820
BFGS:   11 09:38:45      -72.761943        0.0223
BFGS:   12 09:38:45      -72.761945        0.0004
BFGS:   13 09:38:45      -72.761945        0.0000
Volume change of 276.04455743665795 %, as a result of the following lattice parameter changes:
[54.55790938145649, 54.557909381428715, 57.41888723717707

In [11]:
!ase gui LAMMPS.traj

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/thism292/.local/lib/python3.10/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/home/thism292/.local/lib/python3.10/site-packages/ase/gui/pipe.py", line 19, in main
    make_plot(show=False, *data)
  File "/home/thism292/.local/lib/python3.10/site-packages/ase/gui/graphs.py", line 78, in make_plot
    plt.figure(figsize=(basesize * 2.5**0.5, basesize))
  File "/home/thism292/.local/lib/python3.10/site-packages/matplotlib/_api/deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "/home/thism292/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", line 783, in figure
    manager = new_figure_manager(
  File "/home/thism292/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", 

In [12]:
!ase gui LAMMPS_strain.traj

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/thism292/.local/lib/python3.10/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/home/thism292/.local/lib/python3.10/site-packages/ase/gui/pipe.py", line 19, in main
    make_plot(show=False, *data)
  File "/home/thism292/.local/lib/python3.10/site-packages/ase/gui/graphs.py", line 78, in make_plot
    plt.figure(figsize=(basesize * 2.5**0.5, basesize))
  File "/home/thism292/.local/lib/python3.10/site-packages/matplotlib/_api/deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "/home/thism292/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", line 783, in figure
    manager = new_figure_manager(
  File "/home/thism292/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", 

In [1]:
from ccs_fit.scripts.ccs_export_FF import write_FF

write_FF("CCS_params.json")

Writing LAMMPS and GULP splines.


/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/thism292/Documents/repos/CCS_fit/src/ccs_fit/scripts/ccs_export_FF.py:25: RuntimeWarning: overflow encountered in square
  return De*((1-np.exp(-a*(r-re)))**2-1)


RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.